<a href="https://www.kaggle.com/code/lestiessam/tweet-analysis-and-prediction?scriptVersionId=199790884" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import tensorflow as tf
print(tf.version.VERSION)

2.16.1


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


**Install necessary packages**

In [3]:
!pip install datasets
!pip install pandas
!pip install transformers
!pip install evaluate
!pip install numpy
!pip install datasets pandas transformers evaluate numpy
!pip install torch

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 941.7 kB/s eta 0:00:00:010:00:01


**Load the data to use**

In [5]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

**Tokenizer**

In [7]:
from transformers import GPT2Tokenizer

#Loading the dataset to train our model
dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [8]:
small_train_dataset =  tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

**Initialize our base model**

In [9]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2",num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Evaluate Method**

In [10]:
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

**Fine-tune using the Trainer Method**

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "test_trainer",
    #evaluation_strategy = "epoch",
    per_device_train_batch_size =1,  #Reduce batch size here
    per_device_eval_batch_size =1,  #optionally, reduce for
    gradient_accumulation_steps=4
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=375, training_loss=1.1630161946614583, metrics={'train_runtime': 2321.8749, 'train_samples_per_second': 1.292, 'train_steps_per_second': 0.162, 'total_flos': 1567794659328000.0, 'train_loss': 1.1630161946614583, 'epoch': 3.0})

In [13]:
import evaluate
import numpy as np

trainer.evaluate()

{'eval_loss': 1.0481990575790405,
 'eval_accuracy': 0.483,
 'eval_runtime': 62.975,
 'eval_samples_per_second': 15.879,
 'eval_steps_per_second': 7.94,
 'epoch': 3.0}

In [17]:
import torch

# Set up the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device
model = model.to(device)

# Function to tokenize and make prediction
def predict_tweet_sentiment(tweet):
    # Tokenize the tweet and move tensors to the same device as the model
    inputs = tokenizer(tweet, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the correct device
    
    # Get model predictions
    with torch.no_grad():  # Disable gradient computation for faster inference
        outputs = model(**inputs)
    
    # Extract logits and apply softmax to get probabilities
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()  # Get the predicted class (index)
    
    # Map predicted class to sentiment
    sentiment = label_mapping[predicted_class]
    return sentiment

In [27]:
# Example tweet
tweet = "Hello, you are mad"

# Make prediction
predicted_sentiment = predict_tweet_sentiment(tweet)
print(f"Predicted sentiment for tweet: {predicted_sentiment}")

Predicted sentiment for tweet: Negative
